In [1]:
# https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [2]:
import numpy as np
import math
from sklearn.model_selection import train_test_split

In [3]:
math.inf

inf

In [59]:
def gini_index(groups, labels):
    gini = 0.0
    instances_len = float(sum([len(group) for group in groups]))
    
    for group in groups:
        group_size = float(len(group))
        if group_size == 0:
            continue
        score = 0.0
        for class_label in labels:
            p = [row[-1] for row in group].count(class_label) / group_size
            score += p * p 
        gini += (1.0 - score) * (group_size / instances_len)
        
    return gini

In [60]:
assert gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]) == 0.5

In [61]:
def test_split(attribute_index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[attribute_index] < value:
            left.append(row)
        else: 
            right.append(row)
    return left, right

In [62]:
def get_split(dataset):
    class_values = list(set([row[-1] for row in dataset]))
    attribute_index, value, score, groups = 999,999,999, None
    for index in range(len(dataset[0]) - 1):
        for row in dataset:
            split_groups = test_split(index, row[index], dataset)
            gini = gini_index(split_groups, class_values)
            if gini < score:
                attribute_index, value, score, groups = index, row[index], gini, split_groups
                
    return {'attribute_index': attribute_index, 'value': value, 'groups': groups}

In [63]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]

In [64]:
split = get_split(dataset)

In [65]:
split

{'attribute_index': 0,
 'value': 6.642287351,
 'groups': ([[2.771244718, 1.784783929, 0],
   [1.728571309, 1.169761413, 0],
   [3.678319846, 2.81281357, 0],
   [3.961043357, 2.61995032, 0],
   [2.999208922, 2.209014212, 0]],
  [[7.497545867, 3.162953546, 1],
   [9.00220326, 3.339047188, 1],
   [7.444542326, 0.476683375, 1],
   [10.12493903, 3.234550982, 1],
   [6.642287351, 3.319983761, 1]])}

In [66]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [67]:
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

In [68]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

In [69]:
tree = build_tree(dataset, 1,1)

In [70]:
tree

{'attribute_index': 0, 'value': 6.642287351, 'left': 0, 'right': 1}

In [71]:
def predict(node, row):
	if row[node['attribute_index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [72]:
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = []
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return predictions

In [73]:
for row in dataset:
    pred = predict(tree, row)
    print(f"Actual = {row[-1]}. Pred: {pred}")

Actual = 0. Pred: 0
Actual = 0. Pred: 0
Actual = 0. Pred: 0
Actual = 0. Pred: 0
Actual = 0. Pred: 0
Actual = 1. Pred: 1
Actual = 1. Pred: 1
Actual = 1. Pred: 1
Actual = 1. Pred: 1
Actual = 1. Pred: 1


In [74]:
bank_data = np.genfromtxt('data_banknote_authentication.txt', delimiter=',')

In [75]:
bank_data[-1]

array([-2.5419 , -0.65804,  2.6842 ,  1.1952 ,  1.     ])

In [76]:
pred = decision_tree(bank_data, bank_data, 5, 10)

In [77]:
len(pred) == len(bank_data)

True

In [78]:
correct = 0
for i in range(len(pred)):
    if pred[i] == bank_data[i][-1]:
        correct+=1
    
correct / len(bank_data)

0.9839650145772595